**Task 07: Querying RDF(s)**

In [1]:
!pip install rdflib 
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2020-2021/master/Assignment4"

     |████████████████████████████████| 235kB 8.8MB/s 
     |████████████████████████████████| 51kB 6.3MB/s 


Leemos el fichero RDF de la forma que lo hemos venido haciendo

In [2]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS
g = Graph()
g.namespace_manager.bind('ns', Namespace("http://somewhere#"), override=False)
g.namespace_manager.bind('vcard', Namespace("http://www.w3.org/2001/vcard-rdf/3.0#"), override=False)
g.parse(github_storage+"/resources/example6.rdf", format="xml")

<Graph identifier=N8fbf2a0eb22f457382692bb72418e3cd (<class 'rdflib.graph.Graph'>)>

**TASK 7.1: List all subclasses of "Person" with RDFLib and SPARQL**

In [39]:
ns= Namespace("http://somewhere#")
VCARD = Namespace("http://www.w3.org/2001/vcard-rdf/3.0#")
for s,p,o in g.triples((None, RDFS.subClassOf,ns.Person)):
  print(s)
from rdflib.plugins.sparql import prepareQuery

query1 = prepareQuery('''
  SELECT 
    ?Subject
  WHERE { 
    ?Subject rdfs:subClassOf ns:Person. 
  }
  ''',
  initNs = { "rdfs":RDFS , "ns":ns}
)

for r in g.query(query1):
  print(r)

http://somewhere#Researcher
(rdflib.term.URIRef('http://somewhere#Researcher'),)


**TASK 7.2: List all individuals of "Person" with RDFLib and SPARQL (remember the subClasses)**


In [42]:
ns= Namespace("http://somewhere#")

for s, p, o in g.triples((None, RDF.type, ns.Person)):
  print(s) 
for s,p,o in g.triples((None, RDFS.subClassOf, ns.Person)):
  for t,v,y in g.triples((None, RDF.type, s)):
    print(t)
from rdflib.plugins.sparql import prepareQuery
query2 = prepareQuery('''
  SELECT ?s
  WHERE { 
    { 
      ?s rdf:type ns:Person. 
    } UNION {
      ?subclass rdfs:subClassOf ns:Person.
      ?s rdf:type ?subclass
    }
  }
  ''',
  initNs = { "rdfs": RDFS,"rdf": RDF, "ns": ns}
)

for r in g.query(query2):
  print(r)

http://somewhere#SaraJones
http://somewhere#JohnSmith
http://somewhere#JaneSmith
(rdflib.term.URIRef('http://somewhere#SaraJones'),)
(rdflib.term.URIRef('http://somewhere#JohnSmith'),)
(rdflib.term.URIRef('http://somewhere#JaneSmith'),)


**TASK 7.3: List all individuals of "Person" and all their properties including their class with RDFLib and SPARQL**


In [46]:
ns= Namespace("http://somewhere#")
for s,p,o in g.triples((None, RDF.type, ns.Person)):
  for t,v,y in g.triples((s, None, None)):
      print(t,v,y)

for s,p,o in g.triples((None, RDFS.subClassOf, ns.Person)):
  for t,v,y in g.triples((None, RDF.type, s)):
    for a,b,c in g.triples((t, None, None)):
      print(a,b,c)
from rdflib.plugins.sparql import prepareQuery
query3 = prepareQuery('''
  SELECT ?s ?o ?p
  WHERE { 
    { 
      ?s rdf:type ns:Person.
      ?s ?o ?p
    } UNION {
      ?s rdf:type ?t.
      ?s ?o ?p.
      ?t rdfs:subClassOf ns:Person
    }
  }
  ''',
  initNs = { "rdfs": RDFS,"rdf": RDF, "ns": ns}
)

for r in g.query(query3):
  print(r)

http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given Sara
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN Sara Jones
http://somewhere#SaraJones http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Jones
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given John
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN John Smith
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Smith
http://somewhere#JohnSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#JaneSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Researcher
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given Jane
http://somewhere#JaneSmith http://somewhere#h